In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.spectral_norm import spectral_norm
from reversible2.conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
rng = RandomState(394839483)

In [ ]:
a =  rng.rand(200,2)

plt.figure(figsize=(4,4))
plt.plot(a[:,0], a[:,1], marker='o', ls='')
plt.xlim(-2,2)
plt.ylim(-2,2)

In [ ]:
from torch import nn
from reversible2.blocks import dense_add_block

In [ ]:

from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 5001
for i_epoch in range(n_epochs):
    examples = th.rand(200,2)
    out = feature_model(examples)
    loss = th.mean(out * out)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        examples = th.rand(200,2)
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)

In [ ]:
set_random_seeds(3,False)
examples = th.rand(300,2)
a = var_to_np(examples)
b = var_to_np(feature_model(examples))
fig = plt.figure(figsize=(4,4))
plt.plot(a[:,0], a[:,1], marker='o', ls='')
plt.plot(b[:,0], b[:,1], marker='o', ls='')
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.legend(("Inputs", "Encodings"))
plt.title("Loss: Squared L2 norm")
display(fig)
plt.close(fig)

In [ ]:
set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 2001
for i_epoch in range(n_epochs):
    examples = th.rand(200,2)
    out = feature_model(examples)
    loss = th.mean(th.abs(out))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        examples = th.rand(200,2)
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)

In [ ]:
set_random_seeds(3,False)
examples = th.rand(300,2)
a = var_to_np(examples)
b = var_to_np(feature_model(examples))
fig = plt.figure(figsize=(4,4))
plt.plot(a[:,0], a[:,1], marker='o', ls='')
plt.plot(b[:,0], b[:,1], marker='o', ls='')
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.legend(("Inputs", "Encodings"))
plt.title("Loss: L1 norm")
display(fig)
plt.close(fig)

In [ ]:


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-4)

In [ ]:
set_random_seeds(53434, False)
n_epochs = 20001
examples = th.rand(15,2)
for i_epoch in range(n_epochs):
    out = feature_model(examples)
    loss = th.mean(th.abs(out))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)

In [ ]:
from reversible2.invert import invert
outs = feature_model(examples)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)

In [ ]:
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.legend(["Step " +  str(i) for i in range(5)], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")



In [ ]:


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-4)

In [ ]:
set_random_seeds(53434, False)
n_epochs = 20001
examples = th.rand(15,2)
for i_epoch in range(n_epochs):
    out = feature_model(examples)
    loss = th.mean(out * out)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)
        outs = feature_model(examples)
        all_inverted = invert(feature_model, outs, return_all=True)[::-1]
        len(all_inverted)
        fig = plt.figure(figsize=(4.5,4.5))
        for i_inv, this_inv in enumerate(all_inverted):

            pts = var_to_np(this_inv)
            plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
        for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
            for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
                plt.annotate("",
                             xy=one_next, xytext=one_prev,
                           arrowprops=dict(arrowstyle="->",),
                            )

        plt.legend(["Step " +  str(i) for i in range(5)], bbox_to_anchor=(1,1,0,0))
        plt.xlabel("x1")
        plt.ylabel("x2")
        display(fig)
        plt.close(fig)



In [ ]:
a = var_to_np(examples)
b = var_to_np(feature_model(examples))
fig = plt.figure(figsize=(4,4))
plt.plot(a[:,0], a[:,1], marker='o', ls='')
plt.plot(b[:,0], b[:,1], marker='o', ls='')
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.legend(("Inputs", "Encodings"))
plt.title("15 examples dataset\nSquared L2 norm")
display(fig)
plt.close(fig)

In [ ]:
outs = feature_model(examples)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):

    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.legend(["Step " +  str(i) for i in range(5)], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")
display(fig)
plt.close(fig)



In [ ]:
pts = np.linspace(-0.1,1.1,10)

a = np.array([np.stack((np.ones(len(pts)) * p, pts.copy())) for p in pts]).transpose(0,2,1).reshape(-1,2)

plt.figure(figsize=(5,5))
plt.plot(a[:,0], a[:,1], ls='', marker='o')


outs = feature_model(np_to_var(a, dtype=np.float32))

b = var_to_np(outs)
plt.figure(figsize=(5,5))

plt.plot(a[:,0], a[:,1], ls='', marker='o')
plt.plot(b[:,0], b[:,1], ls='', marker='o')
plt.xlim(-4,4)
plt.ylim(-4,4)
for one_a, one_b in zip(a,b):
    plt.plot([one_a[0], one_b[0]], [one_a[1], one_b[1]],
            color='black', lw=0.5)

In [ ]:


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-4)

In [ ]:
set_random_seeds(53434, False)
n_epochs = 20001
examples = th.rand(15,2)
for i_epoch in range(n_epochs):
    out = feature_model(examples)
    perturb = th.rand(out.size())
    scale = 0.001
    scaled = (perturb / th.norm(perturb, dim=1, keepdim=True)) * scale
    perturbed = out + scaled
    inverted = invert(feature_model, perturbed)
    distances = th.norm(inverted - examples, dim=1) / scale
    distance_loss = th.sum(th.clamp_min(distances - 1,0))
    l2_loss = th.mean(out * out)
    loss = distance_loss + l2_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)
        outs = feature_model(examples)
        all_inverted = invert(feature_model, outs, return_all=True)[::-1]
        len(all_inverted)
        fig = plt.figure(figsize=(4.5,4.5))
        for i_inv, this_inv in enumerate(all_inverted):

            pts = var_to_np(this_inv)
            plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
        for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
            for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
                plt.annotate("",
                             xy=one_next, xytext=one_prev,
                           arrowprops=dict(arrowstyle="->",),
                            )

        plt.legend(["Step " +  str(i) for i in range(5)], bbox_to_anchor=(1,1,0,0))
        plt.xlabel("x1")
        plt.ylabel("x2")
        display(fig)
        plt.close(fig)



In [ ]:
pts = np.linspace(-0.1,1.1,10)

a = np.array([np.stack((np.ones(len(pts)) * p, pts.copy())) for p in pts]).transpose(0,2,1).reshape(-1,2)

plt.figure(figsize=(5,5))
plt.plot(a[:,0], a[:,1], ls='', marker='o')


outs = feature_model(np_to_var(a, dtype=np.float32))

b = var_to_np(outs)
plt.figure(figsize=(5,5))

plt.plot(a[:,0], a[:,1], ls='', marker='o')
plt.plot(b[:,0], b[:,1], ls='', marker='o')
plt.xlim(-4,4)
plt.ylim(-4,4)
for one_a, one_b in zip(a,b):
    plt.plot([one_a[0], one_b[0]], [one_a[1], one_b[1]],
            color='black', lw=0.5)